# Notebook: 04_causal_discovery.ipynb
**Purpose:**  
- 標準化済みパネルデータを読み込み、三層因果ネットワークを推定し、エッジリストを出力する

!pip install git+https://github.com/xunzheng/notears.git

In [1]:
# -----------------------
# ライブラリのインポート
# -----------------------
# Notebook: 04_notears_partial_order.ipynb
# ローカルの notears パッケージを Python パスに追加しておく

import sys
import os
import pandas as pd
import numpy as np
import importlib

# 1) カーネル内にすでに読み込まれている 'notears' モジュールを削除
for mod in list(sys.modules):
    if mod.startswith("notears"):
        del sys.modules[mod]

# 2) “notears” フォルダのパスを正しく指定し、sys.path の先頭に挿入
#    例：この Notebook が "your-project/notebooks" にある場合
repo_path = os.path.abspath(os.path.join(os.getcwd(), "..", "notears"))
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

print("sys.path の先頭に追加されたパス:", sys.path[0])

# 3) 改めてローカルの notears.linear をインポートして、正しい関数が読み込まれるか確認
try:
    import notears.linear
    importlib.reload(notears.linear)
    from notears.linear import notears_linear
    print("✅ ローカル版 notears_linear をインポートできました")
    import inspect
    print("関数シグネチャ:", inspect.signature(notears_linear))
except Exception as e:
    print("❌ インポート失敗:", e)

sys.path の先頭に追加されたパス: C:\Users\user\OneDrive\Desktop\climate-energy-causality\notears
✅ ローカル版 notears_linear をインポートできました
関数シグネチャ: (X, lambda1, loss_type, max_iter=100, h_tol=1e-08, rho_max=1e+16, w_threshold=0.3, mask=None)


In [2]:
import sys, os

# 例：ノートブックと同階層に "notears" フォルダがあるとする場合
repo_path = os.path.abspath(os.path.join(os.getcwd(), "..", "notears"))
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

# 既にインポート済みのモジュールをクリア
import importlib
if "notears.linear" in sys.modules:
    del sys.modules["notears.linear"]
if "notears" in sys.modules:
    del sys.modules["notears"]

# 改めてインポートしてどのファイルが来るかを確認
import notears.linear
print("使われている linear.py の場所:", notears.linear.__file__)

使われている linear.py の場所: C:\Users\user\anaconda3\envs\climate-energy-causality\lib\site-packages\notears\linear.py


In [3]:
# -------------------------------
# データの読み込みと変数順の定義
# -------------------------------
# 標準化済みパネルデータを読み込む
panel_path = "../data/processed/panel/panel_southasia_2000_2021_scaled.csv"
df_panel = pd.read_csv(panel_path)

print("読み込んだデータのカラム一覧:")
print(df_panel.columns.tolist())


読み込んだデータのカラム一覧:
['country', 'iso3', 'year', 'mean_wbgt', 'renewable_energy_pct', 'fossil_fuel_pct', 'electricity_per_capita', 'co2_per_capita', 'gdp_per_capita', 'unemployment_rate', 'health_expenditure_pct', 'agri_valueadded_pct', 'urbanization_pct']


In [4]:
# 層順（Partial Order）を厳密に反映する「変数リスト」を定義
numeric_cols = [
    "mean_wbgt",                    # 気候層 (layer 0)

    "renewable_energy_pct",         # エネルギー層 (layer 1)
    "fossil_fuel_pct",
    "electricity_per_capita",
    "co2_per_capita",

    "gdp_per_capita",               # 社会経済層 (layer 2)
    "unemployment_rate",
    "health_expenditure_pct",
    "agri_valueadded_pct",
    "urbanization_pct"
]

# 全て存在するかチェック
missing = [c for c in numeric_cols if c not in df_panel.columns]
if missing:
    raise ValueError(f"以下の列がデータに見つかりませんでした: {missing}")

# 学習行列 X を作成
X = df_panel[numeric_cols].values  # shape = (サンプル数, 変数数)
n, d = X.shape
print(f"X の形状: {X.shape}  (n={n}, d={d})")

X の形状: (110, 10)  (n=110, d=10)


In [16]:
# 国別に mean_wbgt とエネルギー変数の相関をチェックする例
for country, df_ct in df_panel.groupby("country"):
    corr_renew = df_ct["mean_wbgt"].corr(df_ct["renewable_energy_pct"])
    corr_elec  = df_ct["mean_wbgt"].corr(df_ct["electricity_per_capita"])
    corr_co2   = df_ct["mean_wbgt"].corr(df_ct["co2_per_capita"])
    print(f"{country}: corr(mean_wbgt, renewable)={corr_renew:.3f}, corr(mean_wbgt, electricity)={corr_elec:.3f}, corr(mean_wbgt, co2)={corr_co2:.3f}")

Bangladesh: corr(mean_wbgt, renewable)=-0.625, corr(mean_wbgt, electricity)=0.610, corr(mean_wbgt, co2)=0.612
India: corr(mean_wbgt, renewable)=-0.679, corr(mean_wbgt, electricity)=0.754, corr(mean_wbgt, co2)=0.703
Nepal: corr(mean_wbgt, renewable)=-0.349, corr(mean_wbgt, electricity)=0.388, corr(mean_wbgt, co2)=0.365
Pakistan: corr(mean_wbgt, renewable)=-0.577, corr(mean_wbgt, electricity)=0.528, corr(mean_wbgt, co2)=0.571
Sri Lanka: corr(mean_wbgt, renewable)=-0.700, corr(mean_wbgt, electricity)=0.747, corr(mean_wbgt, co2)=0.726


## 南アジア５か国の平均的ネットワーク

In [5]:
# --------------------------
# 「部分的順序マスク」の作成
# --------------------------
# Partial Order を定義 (layer_map を作成)
# 1) 各変数の“層番号”を定義 (0=気候, 1=エネルギー, 2=社会経済)
# ここでは三層：
#   - 気候層: mean_wbgt
#   - エネルギー層: renewable_energy_pct, fossil_fuel_pct, electricity_per_capita, co2_per_capita
#   - 社会経済層: gdp_per_capita, unemployment_rate, health_expenditure_pct, agri_valueadded_pct, urbanization_pct
layer_map = {
    "mean_wbgt": 0,
    "renewable_energy_pct": 1,
    "fossil_fuel_pct": 1,
    "electricity_per_capita": 1,
    "co2_per_capita": 1,
    "gdp_per_capita": 2,
    "unemployment_rate": 2,
    "health_expenditure_pct": 2,
    "agri_valueadded_pct": 2,
    "urbanization_pct": 2
}

# mask 行列 (d x d) を作成。mask[i,j] == 1 は「変数 j→i を禁止」という意味
mask = np.zeros((d, d), dtype=int)
for i, tgt in enumerate(numeric_cols):
    for j, src in enumerate(numeric_cols):
        # src の層 > tgt の層 なら禁止
        if layer_map[src] > layer_map[tgt]:
            mask[i, j] = 1

# mask の一部を表示しておく
print("mask 行列のサンプル (先頭 5 行, 5 列):")
print(mask[:5, :5])

mask 行列のサンプル (先頭 5 行, 5 列):
[[0 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [21]:
# ローカル版 notears_linear を呼び出して学習

# 改めてインポート（既にインポート済みなら reload でもOK）
import importlib
import notears.linear
importlib.reload(notears.linear)
from notears.linear import notears_linear

# 正則化パラメータ λ を設定
lambda1 = 0.1

print("\n-- notears_linear の学習を開始します --")
W_est = notears_linear(
    X,
    lambda1=0.001,
    loss_type="l2",
    max_iter=100,
    h_tol=1e-8,
    rho_max=1e+16,
    w_threshold=1e-8,
    mask=mask
)
print("-- 学習完了 --")
print("返り値 W_est の形状:", W_est.shape)


-- notears_linear の学習を開始します --
-- 学習完了 --
返り値 W_est の形状: (10, 10)


In [22]:
# 結果を DataFrame に変換し、エッジリストとして保存

# W_est を DataFrame に変換 (i 行, j 列 = j→i の重み)
df_W = pd.DataFrame(W_est, index=numeric_cols, columns=numeric_cols)

# 非ゼロエントリだけを表示して確認
print("\n最終的な重み行列 (非ゼロのみ表示):")
print(df_W.replace(0, pd.NA).dropna(how="all", axis=0).dropna(how="all", axis=1))

# エッジリスト作成
edges = []
threshold = 1e-6
for i, tgt in enumerate(numeric_cols):
    for j, src in enumerate(numeric_cols):
        w = df_W.iloc[i, j]
        if abs(w) > threshold:
            edges.append({"source": src, "target": tgt, "weight": float(w)})

df_edges = pd.DataFrame(edges)
print(f"\n抽出されたエッジ数: {len(df_edges)}")
print(df_edges.head())

# CSV 保存
os.makedirs("../data/processed/panel", exist_ok=True)
edges_path = "../data/processed/panel/edges_notears_masked.csv"
df_edges.to_csv(edges_path, index=False)
print(f"エッジリストを保存: {edges_path}")



最終的な重み行列 (非ゼロのみ表示):
                       mean_wbgt renewable_energy_pct fossil_fuel_pct  \
renewable_energy_pct   -0.137042                 <NA>       -0.000041   
fossil_fuel_pct         1.307336             -0.98826            <NA>   
electricity_per_capita -0.000024                 -0.0        0.000012   
co2_per_capita         -0.000018            -0.000001        0.000015   
gdp_per_capita          0.000001                 -0.0        0.000001   
unemployment_rate      -0.000016                 -0.0       -0.000008   
health_expenditure_pct -0.000004                 -0.0       -0.000004   
agri_valueadded_pct     -0.00002                 -0.0       -0.000006   
urbanization_pct       -0.882442            -0.000001        0.000027   

                       electricity_per_capita co2_per_capita gdp_per_capita  \
renewable_energy_pct                -1.075055      -0.008049           <NA>   
fossil_fuel_pct                          <NA>       0.455398           <NA>   
electricity

In [19]:
# ---------------------------
# 層ごとのエッジ制約の確認
# もし、社会経済変数 → 気候変数」のような“逆エッジ”があれば警告を出す
# ---------------------------
# 各変数がどの層に属するかを辞書で定義
layer_map = {
    "mean_wbgt": 0,  # 気候層
    "renewable_energy_pct": 1,
    "fossil_fuel_pct": 1,
    "electricity_per_capita": 1,
    "co2_per_capita": 1,  # エネルギー構造層
    "gdp_per_capita": 2,
    "unemployment_rate": 2,
    "health_expenditure_pct": 2,
    "agri_valueadded_pct": 2,
    "urbanization_pct": 2   # 社会経済層
}

# 逆エッジをチェック
violations = []
for idx, row in df_edges.iterrows():
    src = row["source"]
    tgt = row["target"]
    if layer_map[src] > layer_map[tgt]:
        violations.append((src, tgt, row["weight"]))

print(f"逆エッジの検出数: {len(violations)}")
if violations:
    print("逆エッジ (source → target → weight):")
    for v in violations:
        print(f"  {v[0]} → {v[1]} (weight={v[2]:.4f})")
else:
    print("層構造の逆転エッジは検出されませんでした。")


逆エッジの検出数: 0
層構造の逆転エッジは検出されませんでした。


## 国別 NOTEARS

In [23]:
import pandas as pd
import numpy as np
import os
from notears.linear import notears_linear  # mask付きに改造済み

results = []
for country, df_ct in df_panel.groupby("country"):
    X_ct = df_ct[numeric_cols].values
    # NOTEARS 学習
    W_ct = notears_linear(
        X_ct,
        lambda1=0.001,
        loss_type="l2",
        max_iter=100,
        h_tol=1e-8,
        rho_max=1e16,
        w_threshold=1e-8,
        mask=mask
    )
    # DataFrame にして保存（解析用）
    df_W_ct = pd.DataFrame(W_ct, index=numeric_cols, columns=numeric_cols)
    df_W_ct["country"] = country
    results.append(df_W_ct)

# 結果を結合してひとつの DataFrame にまとめる
df_all_countries = pd.concat(results, ignore_index=False)
# (indexがnumeric_colsの各国分×、country列が付与された状態)

# たとえば、「source,target,weight,country」形式のエッジリストに展開する例
edges = []
for country, df_ct in df_all_countries.groupby("country"):
    for tgt in numeric_cols:
        for src in numeric_cols:
            w = df_ct.at[tgt, src]
            if abs(w) > 1e-6:
                edges.append({
                    "country": country,
                    "source": src,
                    "target": tgt,
                    "weight": float(w)
                })
df_edges_country = pd.DataFrame(edges)
df_edges_country.to_csv("../data/processed/panel/edges_per_country.csv", index=False)

次ステップへのメモ
1. エッジリストをグラフ描画ソフトに読み込む
 - Gephi/Cytoscape などでエッジリスト CSV をインポートし、レイアウト設定やノード属性（層番号や重み）を適用して可視化する。
 - 中心性指標（betweenness, degree, PageRank など）を計算し、政策介入ポイントを検討。

2. do演算による介入シミュレーション
 - もし「mean_wbgt に +1 標準偏差の介入を行ったら、下流の gdp_per_capita はどれくらい変化するか」などを検証したい場合、
   - Python コードで モデルの重み を用いて手動でパス分析を行うか、
   - DoWhy や econml などを使って複雑な介入分析を行う選択肢がある。

3. 層ごとの強調
 - ネットワーク可視化時に「気候層」「エネルギー層」「社会経済層」を色分けし、政策提言に役立つ重要ノードを見つける。